# Implementación Path Relinking
---

In [119]:
from structure import instance, solution
from algorithms import grasp
from structure.solution import evaluate
from copy import deepcopy
import pandas as pd
import random

In [218]:
path = "instances/MDG-a_12_100_m10.txt"
inst = instance.readInstance(path)
sol, conjunto_soluciones, tiempo_final_grasp = grasp.execute(inst, 1, 0.1)

In [219]:
conjunto_soluciones_pd = pd.DataFrame(conjunto_soluciones)
conjunto_soluciones_pd = conjunto_soluciones_pd.drop_duplicates("sol")

In [220]:
conjunto_soluciones_pd.sort_values("of", ascending=False)

,instance,sol,of
9,"{'n': 100, 'p': 10, 'd': [[0, 8.22, 8.64, 0.75...","{64, 96, 13, 15, 51, 20, 86, 55, 59, 93}",354.25
238,"{'n': 100, 'p': 10, 'd': [[0, 8.22, 8.64, 0.75...","{2, 98, 37, 90, 91, 12, 20, 26, 59, 63}",351.82
30,"{'n': 100, 'p': 10, 'd': [[0, 8.22, 8.64, 0.75...","{64, 70, 43, 79, 16, 20, 86, 59, 60, 93}",350.18
164,"{'n': 100, 'p': 10, 'd': [[0, 8.22, 8.64, 0.75...","{5, 70, 39, 8, 74, 43, 14, 79, 17, 60}",349.53
62,"{'n': 100, 'p': 10, 'd': [[0, 8.22, 8.64, 0.75...","{66, 3, 14, 78, 15, 83, 53, 56, 61, 94}",349.45
...,...,...,...
281,"{'n': 100, 'p': 10, 'd': [[0, 8.22, 8.64, 0.75...","{65, 35, 68, 71, 11, 13, 48, 87, 28, 29}",319.17
1,"{'n': 100, 'p': 10, 'd': [[0, 8.22, 8.64, 0.75...","{72, 74, 75, 12, 13, 14, 17, 58, 59, 95}",319.05
420,"{'n': 100, 'p': 10, 'd': [[0, 8.22, 8.64, 0.75...","{67, 26, 72, 40, 75, 81, 85, 22, 24, 58}",317.13
15,"{'n': 100, 'p': 10, 'd': [[0, 8.22, 8.64, 0.75...","{97, 71, 42, 77, 80, 20, 21, 87, 24, 31}",317.04


In [221]:
sol_max = conjunto_soluciones_pd[conjunto_soluciones_pd["of"] == max(conjunto_soluciones_pd["of"])].iloc[0,1]
conjunto_soluciones_pd["n_distintos"] = conjunto_soluciones_pd["sol"].map(lambda x: len(x.difference(sol_max)))
conjunto_soluciones_pd[conjunto_soluciones_pd["n_distintos"] > 0].sort_values("of", ascending=False)

,instance,sol,of,n_distintos
238,"{'n': 100, 'p': 10, 'd': [[0, 8.22, 8.64, 0.75...","{2, 98, 37, 90, 91, 12, 20, 26, 59, 63}",351.82,8
30,"{'n': 100, 'p': 10, 'd': [[0, 8.22, 8.64, 0.75...","{64, 70, 43, 79, 16, 20, 86, 59, 60, 93}",350.18,5
164,"{'n': 100, 'p': 10, 'd': [[0, 8.22, 8.64, 0.75...","{5, 70, 39, 8, 74, 43, 14, 79, 17, 60}",349.53,10
62,"{'n': 100, 'p': 10, 'd': [[0, 8.22, 8.64, 0.75...","{66, 3, 14, 78, 15, 83, 53, 56, 61, 94}",349.45,9
166,"{'n': 100, 'p': 10, 'd': [[0, 8.22, 8.64, 0.75...","{2, 98, 37, 15, 16, 20, 86, 59, 60, 93}",349.36,5
...,...,...,...,...
281,"{'n': 100, 'p': 10, 'd': [[0, 8.22, 8.64, 0.75...","{65, 35, 68, 71, 11, 13, 48, 87, 28, 29}",319.17,9
1,"{'n': 100, 'p': 10, 'd': [[0, 8.22, 8.64, 0.75...","{72, 74, 75, 12, 13, 14, 17, 58, 59, 95}",319.05,8
420,"{'n': 100, 'p': 10, 'd': [[0, 8.22, 8.64, 0.75...","{67, 26, 72, 40, 75, 81, 85, 22, 24, 58}",317.13,10
15,"{'n': 100, 'p': 10, 'd': [[0, 8.22, 8.64, 0.75...","{97, 71, 42, 77, 80, 20, 21, 87, 24, 31}",317.04,9


In [214]:
sol_0 = conjunto_soluciones[14]
sol_1 = conjunto_soluciones[57]

In [215]:
def transicion(sol_0, sol_1):
    best = None
    resultado = 0
    sol_2 = deepcopy(sol_0)
    sol_guia = deepcopy(sol_1)
    sol_guia["sol"] = sol_guia["sol"].difference(sol_0["sol"])
    for s0 in sol_0["sol"]:
        for s1 in sol_guia["sol"]:
            if s1 not in sol_0["sol"]:
                sol_2["sol"].remove(s0)
                sol_2["sol"].add(s1)
                sol_2["of"] = evaluate(sol_2)
                if best is None or sol_2["of"] > best["of"]:
                    anyadido = s1
                    best = deepcopy(sol_2)
                sol_2["sol"].remove(s1)
                sol_2["sol"].add(s0)
    
    sol_guia["sol"].remove(anyadido)

    if len(sol_guia["sol"]) > 1:
        resultado = transicion(best, sol_guia)

        if resultado["of"] < best["of"]:
            resultado = best
    else:
        print(best["sol"])
        resultado = best
    

    return resultado

best = transicion(sol_0, sol_1)

{64, 13, 15, 17, 51, 20, 86, 55, 59, 93}


In [216]:
best["of"]

324.46

In [211]:
sol_0["of"]

354.25

In [217]:
sol_1["sol"]

{17}